Improvement from the first submission:
- Preprocessing:
    - Missing data
        Air_temperature: fill with values from previous days
        Fill others variables
    - Timestamps alignment
    - Target variable : 
        - Remove outliers
        - Log1p transformation
- Feature engineering:
    - Study the correlation between variables
    - Regrouping features
- Predict on subsets of the test set (due to the limitation of memory)

<a href='#1'>1. Data's Overview</a>

<a href='#2'>2. Preprocessing</a>

<a href='#3'>3. Feature engineering</a>

<a href='#4'>4. Simple model</a>

In [48]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots
import datetime # manipulating date formats
import gc

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        


/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/input/ashrae-energy-prediction/building_metadata.csv
/kaggle/input/ashrae-energy-prediction/weather_train.csv
/kaggle/input/ashrae-energy-prediction/weather_test.csv
/kaggle/input/ashrae-energy-prediction/train.csv
/kaggle/input/ashrae-energy-prediction/test.csv


# <a id='1'>1. Data's Overview</a>

## Loading data

In [49]:
building_metadata = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')
train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv', parse_dates=['timestamp'])
test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv', parse_dates=['timestamp'])
weather_train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv', parse_dates=['timestamp'])
weather_test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv', parse_dates=['timestamp'])

In [50]:
#Function to reduction memory usage (Source code from Kaggle)
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

## Data's overview

In [51]:
"""print('Size of train data', train.shape)
print('Size of test data', test.shape)
print('Size of weather_train data', weather_train.shape)
print('Size of weather_test data', weather_test.shape)
print('Size of building_metadata data', building_metadata.shape)"""

"print('Size of train data', train.shape)\nprint('Size of test data', test.shape)\nprint('Size of weather_train data', weather_train.shape)\nprint('Size of weather_test data', weather_test.shape)\nprint('Size of building_metadata data', building_metadata.shape)"

In [52]:
#train.head()

In [53]:
#weather_train.head()

In [54]:
#building_metadata.head()

Check missing data

In [55]:
def missing(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percentage = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    missing_data  = pd.concat([total, percentage], axis=1, keys=['Total', 'Percentage'])
    return missing_data.head(data.shape[1])

In [56]:
#missing(weather_train)

In [57]:
#missing(weather_test)

In [58]:
#missing(building_metadata)

# <a id='2'>2. Preprocessing</a>

## a. Missing data

### Air_temperature

There are only 55 missing values in the weather_train data set.

We fill the missing data with the value from the closest previous day without na, at the same time of the day, of the same site (there's no NA from the same hour in two consecutive days of the same site)

In [59]:
import math
def fill_air_temp(data):
    
    '''Function for filling Na data in air_temperature variable'''
    
    na_index = data[data['air_temperature'].isnull()].index
    temp_key = ['site_id', 'timestamp', 'air_temperature']
    
    for i in na_index:
        site_na = data.loc[i, 'site_id']   
        for j in range(5):
            time_na = data.loc[i,'timestamp'] - datetime.timedelta(days = j+1)  
            ind = data[(data.site_id == site_na) & (data.timestamp == time_na)].index  
            
            if math.isnan(data.loc[ind, 'air_temperature']) == False :
                data.loc[i, 'air_temperature'] = data.loc[ind, 'air_temperature'].values
                break
    
    return data

In [60]:
weather_train = fill_air_temp(weather_train)
weather_test = fill_air_temp(weather_test)

### floor_count and year_built

In [61]:
#According to the above table, 53.41% of year_built and 75.50% 
#of floor_count are not available

#We fill the floor_count's NA by mode = 1 (repsent 7.52% of floor_count values) 
#and the year_built by the mean of it value
building_metadata.fillna({'floor_count':1,'year_built': building_metadata['year_built'].mean()}, inplace = True)
building_metadata['primary_use'] = building_metadata['primary_use'].astype('category')

### Other variables

In [62]:
#Even the weather data don't change more in the futur and the past, so we make a 
#forward and backward filling
# First: transformation of the timestamp into a datetime object 
# Second: sorting by site id then timestamp

weather_train = weather_train.sort_values(by=['site_id', 'timestamp']) 
weather_train.fillna(method = 'bfill', inplace=True, limit = 12) #backfill up to 12 hours
weather_train.fillna(method = 'ffill', inplace = True, limit = 12) #forward fill the missing data up to 12 hours


## Weather test
weather_test = weather_test.sort_values(by=['site_id', 'timestamp']) 
weather_test.fillna(method = 'bfill', inplace=True, limit = 12)
weather_test.fillna(method = 'ffill', inplace=True, limit = 12)



In [63]:
#sns.heatmap(weather_train.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')
#sns.heatmap(weather_test.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [64]:
#Train data
missing_cols = [col for col in weather_train.columns if weather_train[col].isna().any()] 
fill_lib = weather_train.groupby('site_id')[missing_cols].transform('mean')
#stores the mean of each feature for each site id
weather_train.fillna(fill_lib, inplace=True) #for each feature with missing 
#values, fill the missing entry with the mean for that site

#Test data
missing_cols = [col for col in weather_test.columns if weather_test[col].isna().any()] 
fill_lib = weather_test.groupby('site_id')[missing_cols].transform('mean')
#stores the mean of each feature for each site id
weather_test.fillna(fill_lib, inplace=True)
del missing_cols, fill_lib

weather_train.isna().sum()
#weather_test.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

## b. Align the timestamp

The timestamps of the weather data and the train/test data are different: those in the weather data is not in the site's local time. So there's need an alignment before merging these dataset by timestamps.

To see this problem, we plot the air temperature (from weather data) and energy consumption (from train data) by site and hour.
We assume that highest air temperature should appear at around 14:00.

In [65]:
def plot_by_site_by_hour(data, column) :
    '''Plot a variable by site and hour'''
    
    plot_key = ['site_id', 'timestamp']
    col_to_plot = data[plot_key + [column]].copy()
    col_to_plot['hour'] = col_to_plot['timestamp'].dt.hour
    
    c = 1
    plt.figure(figsize=(25, 15))
    for site_id, data_by_site in col_to_plot.groupby('site_id'):
        by_site_by_hour = data_by_site.groupby('hour').mean()
        ax = plt.subplot(4, 4, c)
        plt.plot(by_site_by_hour.index,by_site_by_hour[column],'xb-')
        ax.set_title('site: '+str(site_id))
        c += 1
    return plt.show()

In [66]:
weather = pd.concat([weather_train,weather_test],ignore_index=True)

In [67]:
#plot_by_site_by_hour(weather,'air_temperature')

We see that the peak temperature for most of these site are not around 14:00, some even at night, which doesn't make sense.

In [68]:
#Create dataframe having site_id and meter from train and metadata datasets
building_site_dict = dict(zip(building_metadata['building_id'], building_metadata['site_id']))
site_meter = train[['building_id', 'meter', 'timestamp', 'meter_reading']].copy()
site_meter['site_id'] = site_meter.building_id.map(building_site_dict)
del site_meter['building_id']

#Dataframe with site_id and electrical consumption
site_elec = site_meter[site_meter.meter == 0]

In [69]:
#plot_by_site_by_hour(site_elec, 'meter_reading')
del building_site_dict, site_meter, site_elec
gc.collect()

100

The energy consumption of these sites make sense, which mean that the timestamps are more aligned.

In [70]:
# calculate ranks of hourly temperatures within date and site_id
key = ['site_id', 'timestamp', 'air_temperature']
temp = weather.loc[:,key]
temp['temp_rank'] = temp.groupby(['site_id', temp.timestamp.dt.date])['air_temperature'].rank('average')

# site_id x mean hour rank of temperature within day
mean_rank = temp.groupby(['site_id', temp.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

In [71]:
#the timestamp alignment gap for each site
gap = pd.Series(mean_rank.values.argmax(axis=1) - 14)
gap.index.name = 'site_id'

def timestamp_align(data):
    '''Function to align the timestamp'''
    data['offset'] = data.site_id.map(gap)
    data['timestamp_aligned'] = (data.timestamp - pd.to_timedelta(data.offset, unit='H'))
    data['timestamp'] = data['timestamp_aligned']
    del data['timestamp_aligned']
    return data

In [72]:
weather_train = timestamp_align(weather_train)
weather_test = timestamp_align(weather_test)
del weather, temp, gap
gc.collect()

77

## c. Merge data

In [73]:
train_merge = train.merge(building_metadata, on='building_id', how='left', validate='many_to_one')
train_merge = train_merge.merge(weather_train, on=['site_id', 'timestamp'], how='left', validate='many_to_one')
del train, weather_train

test_merge = test.merge(building_metadata, on='building_id', how='left', validate='many_to_one')
test_merge = test_merge.merge(weather_test, on=['site_id', 'timestamp'], how='left', validate='many_to_one')
del test, weather_test, building_metadata

This additionnal missing values appear here because some merging identity (timesamp) of train and test dataset aren't in  weather data. 

In [ ]:
#train_data
train_merge = train_merge.sort_values(by=['building_id', 'timestamp'])
train_merge.fillna(method = 'ffill', inplace=True)

#test data
test_merge = test_merge.sort_values(by=['building_id', 'timestamp'])
test_merge.fillna(method = 'ffill', inplace=True)

In [ ]:
train_merge = reduce_mem_usage(train_merge)
test_merge = reduce_mem_usage(test_merge)

## d. Create new variables

With the problem of the form of time series, we can create some useful variables. For example: year, month, date, hour, lagged variables. We will start with the simple ones: year, month, date, hour.

In [ ]:
train_merge['month'] = train_merge['timestamp'].dt.month.astype(np.int8)
#train_merge['year'] = train_merge['timestamp'].dt.year.astype(np.int16)
train_merge['day'] = train_merge['timestamp'].dt.dayofweek.astype(np.int8)
train_merge['week'] = train_merge['timestamp'].dt.weekofyear.astype(np.int8)
train_merge['hour'] = train_merge['timestamp'].dt.hour.astype(np.int8)

test_merge['month'] = test_merge['timestamp'].dt.month.astype(np.int8)
#test_merge['year'] = test_merge['timestamp'].dt.year.astype(np.int16)
test_merge['day'] = test_merge['timestamp'].dt.dayofweek.astype(np.int8)
test_merge['week'] = test_merge['timestamp'].dt.weekofyear.astype(np.int8)
test_merge['hour'] = test_merge['timestamp'].dt.hour.astype(np.int8)

In [ ]:
gc.collect()

## e. Target variable and transformation

 Our original target variable is meter_reading.
 - Type 0 (electricity) of meter are mesure in kBTU. We need to convert them to kWh, and convert the prediction of this type again to kBTU before submission.
 - Transform the original target variable to obtain a new target variable *log1p(meter_reading)*

In [ ]:
train_merge.loc[train_merge.meter == 0 , 'meter_reading'] = train_merge.loc[train_merge.meter == 0 , 'meter_reading'] * 0.2931

In [ ]:
#train_merge.meter_reading.describe()

In [ ]:
#plt.boxplot(train_merge.meter_reading)
#plt.show()

In [ ]:
#plt.hist(train_merge.meter_reading, bins = 100)
#plt.show()

In [ ]:
#plt.hist(np.log1p(train_merge.meter_reading), bins = 100)
#plt.show()

### Identify Outliers

In [ ]:
#train_merge['m/s'] = train_merge['meter_reading']/train_merge['square_feet']

In [ ]:
#elec_ms = train_merge[(train_merge.meter==0) & (train_merge.meter_reading > 0)].loc[:,'m/s']

In [ ]:
#elec_ms.describe()

### Create new target variable

In [ ]:
train_merge['target'] = np.log1p(train_merge['meter_reading'])

In [ ]:
train_0 = train_merge[train_merge.target > 0]
Q1 = train_0.target.quantile(0.25)
Q3 = train_0.target.quantile(0.75)
IQR = Q3 - Q1
del train_0
gc.collect()
train_merge = train_merge[train_merge.target <= Q3 + 1.5 * IQR]

# <a id='3'>3. Feature Engineering</a>

Correlation between target variable and other variable

In [ ]:
#corr = train_merge.corr()

Heat map

In [ ]:
#plt.subplots(figsize=(20, 20))
#sns.heatmap(corr, annot = True, mask = np.triu(corr))

In [ ]:
train_merge.loc[:,'primary_use'] = train_merge['primary_use'].replace({'Entertainment/public assembly':'public',
     'Public services':'service','Lodging/residential':'resident','Healthcare':'resident',
     'Parking':'parking','Warehouse/storage': 'parking','Manufacturing/industrial': 'manufact',
     'Retail':'service', 'Services':'service', 'Technology/science':'Office',
     'Food sales and service':'service','Utility':'service','Religious worship':'public'})

test_merge.loc[:,'primary_use'] = test_merge['primary_use'].replace({'Entertainment/public assembly':'public',
     'Public services':'service','Lodging/residential':'resident','Healthcare':'resident',
     'Parking':'parking','Warehouse/storage': 'parking','Manufacturing/industrial': 'manufact',
     'Retail':'service', 'Services':'service', 'Technology/science':'Office',
     'Food sales and service':'service','Utility':'service','Religious worship':'public'})

# <a id='4'>4. Simple model</a>

In [ ]:
from sklearn.tree import DecisionTreeRegressor

y_train = train_merge['target']
x_train = train_merge.dropna(axis = 1)

x = pd.get_dummies(x_train[['primary_use']],drop_first=True)
x_train = pd.concat([x_train, x], axis = 1)
x_train = x_train.drop(['primary_use','meter_reading','target','timestamp',"sea_level_pressure", "wind_direction", "wind_speed"], axis = 1)

In [ ]:
x_test = test_merge.dropna(axis = 1)
x = pd.get_dummies(x_test[['primary_use']],drop_first=True)
x_test = pd.concat([x_test, x], axis = 1)
x_test = x_test.drop(['primary_use','timestamp',"sea_level_pressure", "wind_direction", "wind_speed"], axis = 1)
x_test = x_test.drop('row_id', axis = 1)

In [ ]:
del train_merge,x
gc.collect()


In [ ]:
tree_model = DecisionTreeRegressor(min_samples_split = 200, min_samples_leaf = 100)
tree_model = tree_model.fit(x_train, y_train)

# Light Gradient Boost Machine

In [ ]:
#import lightgbm as lgb
#from sklearn.model_selection import train_test_split
#import random
#random.seed(0)

In [ ]:
"""#X_train, X_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2)
X_train1 = x_train[:int(x_train.shape[0] / 2)]
X_train2 = x_train[int(x_train.shape[0] / 2):]
del x_train

y_train1 = y_train[:int(y_train.shape[0]/2)]
y_train2 = y_train[int(y_train.shape[0] / 2):]
del y_train

#categorical_features = ['building_id', 'site_id', 'meter','month','day','week','hour']

X_train1 = lgb.Dataset(X_train1, label=y_train1,free_raw_data=False)
X_train2 = lgb.Dataset(X_train2, label=y_train2,free_raw_data=False)
del y_train1,y_train2


params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 40,
    "learning_rate": 0.05,
    "feature_fraction": 0.85,
    "reg_lambda": 2,
    "metric": "rmse",
    "force_col_wise": True
}

#Building model with first half and validating on second half:
model1 = lgb.train(params, train_set=X_train1, num_boost_round=1000, valid_sets=[X_train1,X_train2],verbose_eval=200, early_stopping_rounds=200)

#Building model with second half and validating on first half:
model2 = lgb.train(params, train_set=X_train2, num_boost_round=1000, valid_sets=[X_train2,X_train1], verbose_eval=200,early_stopping_rounds=200)
del X_train1,X_train2"""

In [ ]:
ratio = int(len(x_test)/10)
y_pred = np.empty(len(x_test))
for i in range(10):
    #y_pred[i*ratio:(i+1)*ratio] = np.expm1(model1.predict(x_test.iloc[i*ratio:(i+1)*ratio],num_iteration=model1.best_iteration))/2
    #y_pred[i*ratio:(i+1)*ratio] += np.expm1(model2.predict(x_test.iloc[i*ratio:(i+1)*ratio],num_iteration=model2.best_iteration))/2
    y_pred[i*ratio:(i+1)*ratio] = np.expm1(tree_model.predict(x_test.iloc[i*ratio:(i+1)*ratio]))
del tree_model
gc.collect()    

In [ ]:
y_pred[x_test.meter==0] /= 0.2931
my_submission = pd.DataFrame({'row_id': test_merge.row_id, 'meter_reading': y_pred})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
y_pred